In [1]:
import pickle

In [2]:
with open('book_weights_sample.pkl', 'rb') as f:
    book_weights_dic = pickle.load(f)

The book weights dictionary is organized as follows: the keys are book ids and the values are the weight for each topic. For the example below, book 3 has a weight of 0.63541293 on topic 14, 0.042711955 on topic 16, etc. Topics are numbered from 0 to 49 (50 topics in total, this number might change). 

In [26]:
book_weights_dic[3]

[(14, 0.63541293), (16, 0.042711955), (30, 0.31054646)]

Next, we load the distance between each tweeter topic and each book topic

In [27]:
NUM_TOPICS = 50
handle = "twitter_handle1"
with open('GoodReads/' + str(NUM_TOPICS) + '_Topics/' + handle + '_topic_dist_' + str(NUM_TOPICS) + '.pkl', 'rb') as f:
    H1_topic_similarities = pickle.load(f)

Here topic similarities contains the similarity between each tweet topic and each book topic. The key is the index for tweet topic. 

In [28]:
H1_topic_similarities.keys()

dict_keys([0, 1, 2, 3, 4])

From example, below topic similarity between the first tweet topic and all the book topics, the first cell is the similarity between tweet topic 0 and book topic 0, etc. 

In [29]:
H1_topic_similarities[0]

array([0.63115036, 0.5410966 , 0.        , 0.60161729, 0.60287399,
       0.37091192, 0.57781857, 0.        , 0.61218628, 0.53073806,
       0.59359596, 0.75747244, 0.63990171, 0.64931094, 0.59167339,
       0.72260887, 0.58345033, 0.57992813, 0.        , 0.65580549,
       0.55813228, 0.59733332, 0.52914746, 0.67388694, 0.        ,
       0.64509788, 0.63094447, 0.61634558, 0.6093578 , 0.61839498,
       0.64139237, 0.39989721, 0.75945939, 0.50196681, 0.50925275,
       0.67151635, 0.61885013, 0.56887963, 0.49521495, 0.67566146,
       0.56198947, 0.57937586, 0.79542029, 0.61065288, 0.        ,
       0.        , 0.56771489, 0.68436507, 0.62473132, 0.64290289])

Next, we need to calcuate the average similarity between each twitter topic and each book!

In [30]:
def book_recommend(topic_similarities, topk = 3):
    TOP = topk # Reccomending the top 5 books

    tweet_recommended = {}
    for tweet_topic in topic_similarities.keys():
        book_ave_similarity_dic = {}
        for book_id in book_weights_dic.keys():
            book_weights = book_weights_dic[book_id]
            ave_similarity = 0
            for book_topic, weight in book_weights:
                ave_similarity += weight *  topic_similarities[tweet_topic][book_topic]
            book_ave_similarity_dic[book_id] = ave_similarity
        ## The code below is very ugly, it is essentiall sorte by value and take topk in Spark...
        sorted_sim_dic = sorted(book_ave_similarity_dic.items(), key = lambda kv: kv[1], reverse = True) # This is essentially a sort by value in pyspark, probably where we can save the most time
        top_books = sorted_sim_dic[:TOP]
        tweet_recommended[tweet_topic] = top_books
    return(tweet_recommended)

In [31]:
recommended_bookids = book_recommend(H1_topic_similarities)

It returns a list of books it recommends top 3 books based on each topic

One can then check what each of the book is.

In [16]:
recommended_bookids

{0: [(7967, 0.536600805949041),
  (8648, 0.5365609490238615),
  (9532, 0.5365146470126313)],
 1: [(32542, 0.8258355597474107),
  (24192, 0.8246464562975124),
  (425029, 0.824095562715859)],
 2: [(11127, 0.4682288891924464),
  (65641, 0.46814034673338345),
  (84119, 0.4680836963448835)],
 3: [(4374400, 0.5490727661632726),
  (8492825, 0.5490133911965749),
  (6931356, 0.5488454908893073)],
 4: [(64216, 0.653977687619366),
  (386372, 0.6207194534347562),
  (34497, 0.600169278863835)]}